In [12]:
import os
import json

from ai21 import AI21Client
from ai21.models.chat import ChatMessage

In [15]:
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from job_search_application.minsearch import Index

In [3]:
client = AI21Client()

In [4]:
def suggest_product_title():
    response = client.chat.completions.create(
        model="jamba-instruct-preview",  # Latest model
        messages=[ChatMessage(   # Single message with a single prompt
            role="user",
            content="Please return only 5 words."
    )],
        temperature=0.8,
        max_tokens=200 # You can also mention a max length in the prompt "limit responses to twenty words"
    )
    print(response.choices[0].message.content)

In [7]:
def a21_first_test():
    response = client.chat.completions.create(
        model="jamba-1.5-mini",  # Latest model
        messages=[ChatMessage(   # Single message with a single prompt
            role="user",
            content="Please return only 5 words."
    )],
        temperature=0.8,
        max_tokens=200 # You can also mention a max length in the prompt "limit responses to twenty words"
    )
    print(response.choices[0].message.content)
    return response


In [8]:
resp = a21_first_test()

Customizable, predictive, intuitive, scalable, collaborative.


In [9]:
resp

ChatCompletionResponse(id='chat-54be4d21edb748108870717cabdb4c63', choices=[ChatCompletionResponseChoice(index=0, message=AssistantMessage(role='assistant', content='Customizable, predictive, intuitive, scalable, collaborative.', tool_calls=None), logprobs=None, finish_reason='stop')], usage=UsageInfo(prompt_tokens=17, completion_tokens=12, total_tokens=29))

In [13]:
with open('../data/job_data.json', 'r') as json_file:
    job_data = json.load(json_file)

In [16]:
index = Index(
    text_fields=[
        "title",
        "company",
        "locations",
        "skills",
        "posted_at",
        "is_remote",
        "snippet_fragments",
        "description"
    ],
    keyword_fields=["id"]
)

index.fit(job_data)

In [17]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=3
    )

    return results

In [18]:
query = "I am looking for job as a CRM Lead in Berlin."
search_results = search(query)

In [10]:
prompt_template = """
You're an expert application coach. Answer the QUESTION based on the CONTEXT from the job database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
job_title: {title}
company_name: {company}
work locations: {locations}
highlighted skills: {skills}
date of posting: {posted_at}
short job summary: {snippet_fragments}
detailed job description: {description}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [30]:
# print(build_prompt(query, search_results))

In [24]:
def llm(prompt, model='jamba-1.5-mini'):
    response = client.chat.completions.create(
    model=model,  
    messages=[ChatMessage(   
        role="user",
        content=prompt
    )],
        temperature=0.8,
        max_tokens=200
    )    
    return response.choices[0].message.content, response

In [25]:
def rag(query, model='jamba-1.5-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer, resp = llm(prompt, model=model)
    return answer, resp

In [26]:
answer, resp = rag(query)

In [27]:
answer

'Given the job title "CRM Lead" and the requirement for experience in CRM, this job posting at FORMEL SKIN in Berlin is highly relevant.'

In [28]:
resp

ChatCompletionResponse(id='chat-e1d85d256c154d89848a6ebc9629f0e5', choices=[ChatCompletionResponseChoice(index=0, message=AssistantMessage(role='assistant', content='Given the job title "CRM Lead" and the requirement for experience in CRM, this job posting at FORMEL SKIN in Berlin is highly relevant.', tool_calls=None), logprobs=None, finish_reason='stop')], usage=UsageInfo(prompt_tokens=5896, completion_tokens=33, total_tokens=5929))